# Q1

## (1)

Use DataFrame API to generate a report of the distribution of customers by the number of orders they have made, including customers who have no record of orders.  The output should be displayed in descending order by numbers of customers first, then in descending order by numbers of orders per customer made. 

In [2]:
dfCustomer = spark.read.csv('../data/customer.csv', header=True, inferSchema=True)
dfOrders = spark.read.csv('../data/orders.csv', header=True, inferSchema=True)

In [51]:
#dfCustomer.printSchema()
#dfCustomer.show(1)

In [52]:
#dfOrders.printSchema()
#dfOrders.show()

In [30]:
df = dfCustomer.join(dfOrders, 'CUSTKEY', 'left_outer').select('CUSTKEY', 'ORDERKEY')
#df.count()

In [86]:
df1 = df.groupBy('CUSTKEY').count().withColumnRenamed('count', 'oCount')
df2 = df.join(df1, 'CUSTKEY', 'left_outer')
#df2.count()

In [87]:
#df2.orderBy('oCount', ascending=False).show(5)

In [100]:
df3 = df2.select('ORDERKEY','oCount')\
.groupBy('oCount')\
.count().withColumnRenamed('count', 'cCount').orderBy(['cCount','oCount'],ascending=False)\
.show(1)

+------+------+
|oCount|cCount|
+------+------+
|    20|  1100|
+------+------+
only showing top 1 row



## (2) 

In the example relational database used in the lectures, each customer is served by a salesperson, which is recorded in the "SalesPerson" column in the "Customer" table (see the database schema below).

Use DataFrame API to generate a report of each salesperson's performance, which is the total net price of his/her customer's orders.  Note that the an order consists of multiple order details, and the net price of an order detail is UnitPrice * OrderQty * (1 - UnitPriceDiscount).

All salespeople should be reported, even if they have 0 order.

In [102]:
dfCustomer = spark.read.csv('../data/Customer.csv', header=True, inferSchema=True)
dfDetail = spark.read.csv('../data/SalesOrderDetail.csv', header=True, inferSchema=True)
dfHeader = spark.read.csv('../data/SalesOrderHeader.csv', header=True, inferSchema=True)

In [128]:
#dfCustomer.printSchema()
#dfDetail.printSchema()
#dfHeader.printSchema()

In [129]:
df1 = dfDetail.select('*', (dfDetail.UnitPrice*dfDetail.OrderQty*(1-dfDetail.UnitPriceDiscount)).alias('netprice'))\
        .groupBy('SalesOrderID').sum('netprice') \
        .withColumnRenamed('sum(netprice)', 'TotalPrice')\
        #.show(3)

In [130]:
df2 = dfHeader.join(df1, 'SalesOrderID', 'left_outer').select('CustomerID', 'TotalPrice')\
.groupBy('CustomerID').sum('TotalPrice')\
.withColumnRenamed('sum(TotalPrice)', 'CustomerPrice')

In [131]:
df3 = dfCustomer.join(df2, 'CustomerID', 'left_outer').groupBy('SalesPerson').sum('CustomerPrice')\
.withColumnRenamed('sum(CustomerPrice)', 'Performance').show()

+--------------------+------------------+
|         SalesPerson|       Performance|
+--------------------+------------------+
|adventure-works\l...|170064.83797000002|
|adventure-works\p...|              null|
|adventure-works\d...|              null|
|adventure-works\shu0|112646.20642399997|
|adventure-works\m...|              null|
|adventure-works\j...|              null|
|adventure-works\j...|              null|
|adventure-works\jae0|     426024.108664|
|adventure-works\g...|              null|
+--------------------+------------------+



# Q2

## (1)

Given a 2-dimensional point set, its Skyline is a point set defined as those points which are not dominated by any other point.  Here, a point (x1,y1) dominates another point (x2,y2) if x1 >= x2 and y1 >= y2.

Use divide-and-conquer to find the Skyline of the given point set.  For each test case, we guarantee that all points (x,y) have x > 0 and y > 0. The skeleton code is provided below.

In [132]:
numPartitions = 10

points = sc.textFile('../data/points.txt',numPartitions)
pairs = points.map(lambda l: tuple(l.split()))
pairs = pairs.map(lambda pair: (int(pair[0]),int(pair[1])))
pairs.cache()
# YOUR CODE BEGINS HERE

PythonRDD[832] at RDD at PythonRDD.scala:53

In [151]:
sortedPairs = pairs.sortByKey(ascending=False)

In [152]:
sortedPairs.take(10)

[(99990, 53983),
 (99989, 21634),
 (99978, 19775),
 (99977, 59457),
 (99969, 11482),
 (99958, 59581),
 (99958, 26054),
 (99956, 79605),
 (99949, 52743),
 (99942, 21085)]

In [191]:
def f(i):
    y = 0
    for x in i:
        if x[1] > y:
            y = x[1]
            yield x
localresults = sortedPairs.mapPartitions(f).collect()  

In [192]:
result = []
y = 0
for x in localresults:
    if x[1] > y:
        y = x[1]
        result.append(x)

In [193]:
result

[(99990, 53983),
 (99977, 59457),
 (99958, 59581),
 (99956, 79605),
 (99885, 83645),
 (99827, 89705),
 (99740, 98217),
 (99662, 98973),
 (96815, 99398),
 (96167, 99795),
 (92693, 99960),
 (92580, 99974),
 (59920, 99992),
 (45834, 100000)]

In [195]:
pairs = sc.parallelize([(1, 10), (2, 10), (3, 10), (4, 10), (5, 10), (6, 10), (7, 10), (8, 10), (9, 10), (10, 10), (11, 10)],numPartitions)
localresults = pairs.sortByKey(ascending=False).mapPartitions(f).collect()
result = []
y = 0
for x in localresults:
    if x[1] > y:
        y = x[1]
        result.append(x)
result

[(11, 10)]

## (2) 

In social networks, "zombies" are users that follow many other users, but are not followed by anybody. Given a graph representing a social network, find the largest zombie, i.e., the user who follows the most other users but is not followed by anybody, and output all users it follows.

In [207]:
sc.addPyFile("/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/jars/graphframes-0.8.0-spark3.0-s_2.12.jar")
from graphframes import *
# Vertices DataFrame
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("c", "a", "friend"),
 ("b", "c", "follow"),
 ("d", "a", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("f", "b", "follow"),
 ("c", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "d", "friend"),
 ("g", "c", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow")
], ["src", "dst", "relationship"])
g = GraphFrame(v, e)
e1 = g.edges.filter("relationship = 'follow'")
v1 = e1.groupBy('dst').count().select('dst')
v2 = e1.groupBy('src').count().orderBy('count', ascending=False)
v3 = v2.select('src').subtract(v1)
v4 = v3.join(v2,'src').first()
e.filter(e['src']==v4.src).select('dst').show()

+---+
|dst|
+---+
|  c|
|  d|
|  b|
+---+



# Q3

## (1)

Load it into spark and use divide-and-conquer to find the number of points located in a given triangle ABC. All the points are in 2 dimensions. The skeleton code is provided below.  

In [219]:
numPartitions = 10
A = (0,0)
B = (10000,0)
C = (0,10000)

points = sc.textFile('../data/points.txt',numPartitions)
pairs = points.map(lambda l: tuple(l.split()))
pairs = pairs.map(lambda pair: (int(pair[0]),int(pair[1])))
pairs.cache()

PythonRDD[1385] at RDD at PythonRDD.scala:53

In [220]:
pairs.take(4)

[(1915, 54625), (83327, 40420), (87760, 43005), (116, 57502)]

In [223]:
def f(i):
    for x in i:
        if x[0]+x[1] < 10000:
            yield x
p = pairs.mapPartitions(f).collect()

In [224]:
len(p)

41

In [226]:
def CrossProduct(x, y) :
    return x[0]*y[1] - y[0]*x[1]
    
def VectorMinus(x, y) :
    return (x[0]-y[0], x[1]-y[1])
from operator import add
def f(iterator):
    result = 0
    for P in iterator :
        PA = VectorMinus(A,P)
        PB = VectorMinus(B,P)
        PC = VectorMinus(C,P)
        ab = CrossProduct(PA,PB)
        bc = CrossProduct(PB,PC)
        ca = CrossProduct(PC,PA)
        if (ab * bc >= 0 and ab * ca >= 0):
            result = result + 1
    yield result
    
result = pairs.mapPartitions(f)
result = result.reduce(add)
print(result)

41


## (2)

In social networks, the service provider will recommend possible friends for you. If a user and you have at least one common friend, and he is following you, then he might be your friend.  Note that "friend" relationships are two-way (i.e., it is guaranteed that if there is a 'friend' edge from A to B, there must be an 'friend' edge from B to A), while "follow" relationships are one-way.

Given the following graph representing a social network, find all possible friends for all users. Your program should output all pairs of users (A, B), where B is recommended to A.  In the output, report the users using their names.

In [229]:
from graphframes import *
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])

# Edges DataFrame
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("a", "c", "friend"),
 ("a", "g", "friend"),
 ("b", "c", "friend"),
 ("c", "a", "friend"),
 ("c", "b", "friend"),
 ("c", "d", "follow"),
 ("c", "g", "friend"),
 ("d", "a", "follow"),
 ("d", "g", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow"),
 ("f", "b", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "c", "friend"),
 ("g", "d", "friend")
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

In [243]:
g.find("(a)-[e1]->(c); (b)-[e2]->(c); (b)-[e3]->(a)")\
.where("e1.relationship='friend'")\
.where("e2.relationship='friend'")\
.where("e3.relationship='follow'")\
.select('a.name', 'b.name')\
.show()

+-----+-------+
| name|   name|
+-----+-------+
|David|Charlie|
|Alice|  David|
|  Bob|  Alice|
+-----+-------+



In [270]:
# Q2
from pyspark.sql.functions import *
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("a", "c", "friend"),
 ("a", "g", "friend"),
 ("b", "c", "friend"),
 ("c", "a", "friend"),
 ("c", "b", "friend"),
 ("c", "d", "follow"),
 ("c", "g", "friend"),
 ("d", "a", "follow"),
 ("d", "b", "follow"),
 ("d", "g", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow"),
 ("f", "b", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "c", "friend"),
 ("g", "d", "friend")
], ["src", "dst", "relationship"])

In [406]:
e1 = e.filter(e.relationship=='follow').groupBy('src').count()\
.withColumnRenamed('src', 'id')\
.withColumnRenamed('count', 'c1')
e2 = e.filter(e.relationship=='follow').groupBy('dst').count()\
.withColumnRenamed('dst', 'id')\
.withColumnRenamed('count', 'c2')
c1 = e2.join(e1, 'id')\
.select('id', (col('c2')>col('c1')).alias('condition1'))

In [407]:
num = e.filter(e.relationship=='follow')\
.groupBy('src')\
.count().count()
avg = e.filter(e.relationship=='follow')\
.groupBy('src')\
.count().select('count', lit(1).alias('s'))\
.groupBy('s').sum('count')\
.select('s', (col('sum(count)')/num))\
.withColumnRenamed('(sum(count) / 5)', 'avg')
c2 = e.filter(e.relationship=='follow')\
.groupBy('src')\
.count().select('src', 'count', lit(1).alias('s'))\
.join(avg, 's').select('src','count', 'avg')\
.select('src', ((col('count')>col('avg')).alias('condition2')))\
.withColumnRenamed('src', 'id')
#c1.show()
#c2.show()
c = c1.join(c2, 'id').filter("condition1 = 'true'").filter("condition2 = 'true'")
c.join(v, 'id').select('name').show()

In [415]:
c.join(v, 'id').select('name').show()

+-----+
| name|
+-----+
|David|
+-----+



In [416]:
# Q3
from graphframes import *
g = GraphFrame(v, e)

In [454]:
collect = g.find("(a)-[e1]->(b); (a)-[e2]->(c); (b)-[e3]->(c)")\
.where("e1.relationship='friend'")\
.where("e2.relationship='friend'")\
.where("e3.relationship='friend'")\
.select('a.name', 'b.name', 'c.name')
#collect.show()

In [470]:
collect.show()

+-------+-------+-------+
|   name|   name|   name|
+-------+-------+-------+
|  Gabby|  Alice|Charlie|
|  Alice|Charlie|  Gabby|
|  Gabby|Charlie|  Alice|
|Charlie|  Alice|  Gabby|
|Charlie|  Gabby|  Alice|
|  Alice|  Gabby|Charlie|
+-------+-------+-------+



In [ ]:
# Q1
array = [1, 2, 1, 1, 2, 2, 1, 1]
s = 4

from pyspark import SparkContext
RDD = sc.parallelize(array, 2).cache()

def sequential(nums: List[int]):
    # FILL IN YOUR CODE HERE

def divide(it):
    # FILL IN YOUR CODE HERE

def conquer(X):
    # FILL IN YOUR CODE HERE

X = RDD.mapPartitions(divide).collect()
result = conquer(X)

In [254]:
array = [1, 2, 1, 1, 2, 2, 1, 1]
s = 4

from pyspark import SparkContext
RDD = sc.parallelize(array, 2).cache()

In [ ]:
def divide(it):
    yield len(list(it))
def conquer(X):
    for i in range(x):
        ss = x[i]
        collect = [x[i]]
        for j in range(x-1)+1:
            ss += x[j]
            collect.append(i)
            if ss == s:
                yield collect
                break
            if ss > s:
                break
            